🔍 Step 1: 레이아웃 자동 탐지 비교 전략
| 항목    | 기존 방식 (`pdfplumber`)      | 제안 방식 (`layout-parser + PyMuPDF`) |
| ----- | ------------------------- | --------------------------------- |
| 추출 기준 | `extract_words()`로 좌우 나누기 | 텍스트 박스(layout element)로 좌우 자동 분리  |
| 기준점   | `x0 < midpoint` 직접 계산     | 자동 박스 좌표 기반으로 클러스터링               |
| 한계    | 글자 밀도/줄바꿈에 따라 라인 깨짐       | 레이아웃 알고리즘이 덜 깨짐                   |
| 장점    | 구조가 단순, 빠름                | 위치 정밀도 우수, 다양한 포맷 대응              |

✅ 테스트 항목 정의
| 항목      | 목표              | 방식                                 |
| ------- | --------------- | ---------------------------------- |
| **1단계** | 신구조문대비표 페이지 감지  | `normalize_text()`로 키워드 기반         |
| **2단계** | 좌우 텍스트 블록 분리    | `pdfplumber` vs `layout-parser` 비교 |
| **3단계** | 조문 감지 (제x조)     | 정규표현식 유연화 후 적용                     |
| **4단계** | 단일 PDF 처리 결과 비교 | 추출된 row 수, 포맷 비교                   |


🧪 단위 테스트 전략
### 📄 단위 테스트 계획 (Notion용 Mermaid 예시)
```mermaid
graph TD
    A[PDF 파일 1개 선택] --> B1[pdfplumber 방식 추출]
    A --> B2[layout-parser 방식 추출]
    B1 --> C1[조문 수 카운트]
    B2 --> C2[조문 수 카운트]
    C1 --> D[결과 비교]
    C2 --> D
    D --> E{어느 쪽이 안정적?}
```


## ✅ Step 1: 신구 대비표 페이지 자동 탐지 + 좌우 분리 (2방식) 단위 테스트 코드   
🎯 목표
 - PDF에서 "신·구조문대비표" 페이지 감지

 - 감지된 페이지에서 좌측(현행), 우측(개정안) 텍스트를 각각 추출

 - 방식 1: pdfplumber

 - 방식 2: layoutparser + PyMuPDF

### ✅ 방식 1: pdfplumber 기반 좌우 블록 분리

In [1]:
import pdfplumber
from collections import defaultdict

def extract_with_pdfplumber(file_path, start_keyword="신구조문대비표", line_tol=2):
    result_rows = []
    capturing = False

    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            if not capturing and start_keyword in text:
                capturing = True

            if capturing:
                words = page.extract_words()
                if not words:
                    continue
                midpoint = page.width / 2
                line_map = defaultdict(lambda: {"left": "", "right": ""})

                for word in words:
                    top_key = round(word['top'] / line_tol) * line_tol
                    if word['x0'] < midpoint:
                        line_map[top_key]["left"] += " " + word['text']
                    else:
                        line_map[top_key]["right"] += " " + word['text']

                for top in sorted(line_map.keys()):
                    l = line_map[top]['left'].strip()
                    r = line_map[top]['right'].strip()
                    if l or r:
                        result_rows.append([l, r])
    return result_rows


### ✅ 방식 2: layoutparser + PyMuPDF 기반 텍스트 박스 좌우 분리

In [2]:
import layoutparser as lp
import fitz  # PyMuPDF

def extract_with_layoutparser(file_path, start_keyword="신구조문대비표", line_tol=15):
    result_rows = []
    model = lp.Detectron2LayoutModel(
        config_path='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
        label_map={0: "Text"},
        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8]
    )

    doc = fitz.open(file_path)
    for page_num, page in enumerate(doc):
        text = page.get_text()
        if start_keyword not in text:
            continue

        pix = page.get_pixmap(dpi=200)
        image = lp.io.load_image(pix.samples, pix.width, pix.height)
        layout = model.detect(image)

        left_text, right_text = "", ""
        midpoint = image.shape[1] / 2

        for block in layout:
            if block.type != "Text":
                continue
            block_text = block.text.strip()
            if block_text:
                if block.block.x_1 < midpoint:
                    left_text += block_text + "\n"
                else:
                    right_text += block_text + "\n"

        result_rows.append([left_text.strip(), right_text.strip()])
        break  # 첫 발견 페이지만 추출
    return result_rows


### ✅ 전체 순회 테스트 코드

```
pip install layoutparser[layoutmodels] pdfplumber pymupdf opencv-python
```

In [3]:
from pathlib import Path

# 🔧 경로 설정
pdf_dir = Path("../data/no_upload")  # PDF가 있는 폴더
pdf_files = pdf_dir.glob("*.pdf")

# ✅ 순회 처리 함수
def run_test_on_all_pdfs():
    for file_name in pdf_files:
        path = str(pdf_dir / file_name)
        print(f"\n📄 파일: {file_name}")

        try:
            plumber_result = extract_with_pdfplumber(path)
            print(f"✅ PDFPlumber 추출 성공 - 행 수: {len(plumber_result)}")
        except Exception as e:
            print(f"❌ PDFPlumber 실패: {e}")

        try:
            layout_result = extract_with_layoutparser(path)
            print(f"✅ LayoutParser 추출 성공 - 행 수: {len(layout_result)}")
        except Exception as e:
            print(f"❌ LayoutParser 실패: {e}")

# 🔁 테스트 실행
run_test_on_all_pdfs()


📄 파일: ..\data\no_upload\2205429_의사국 의안과_의안원문.pdf
❌ PDFPlumber 실패: [Errno 2] No such file or directory: '..\\data\\no_upload\\..\\data\\no_upload\\2205429_의사국 의안과_의안원문.pdf'
❌ LayoutParser 실패: module layoutparser has no attribute Detectron2LayoutModel

📄 파일: ..\data\no_upload\2207157_의사국 의안과_의안원문.pdf
❌ PDFPlumber 실패: [Errno 2] No such file or directory: '..\\data\\no_upload\\..\\data\\no_upload\\2207157_의사국 의안과_의안원문.pdf'
❌ LayoutParser 실패: module layoutparser has no attribute Detectron2LayoutModel

📄 파일: ..\data\no_upload\2208369_의사국 의안과_의안원문.pdf
❌ PDFPlumber 실패: [Errno 2] No such file or directory: '..\\data\\no_upload\\..\\data\\no_upload\\2208369_의사국 의안과_의안원문.pdf'
❌ LayoutParser 실패: module layoutparser has no attribute Detectron2LayoutModel

📄 파일: ..\data\no_upload\2208659_의사국 의안과_의안원문.pdf
❌ PDFPlumber 실패: [Errno 2] No such file or directory: '..\\data\\no_upload\\..\\data\\no_upload\\2208659_의사국 의안과_의안원문.pdf'
❌ LayoutParser 실패: module layoutparser has no attribute Detectron2LayoutMo

In [4]:
from pathlib import Path

# ✅ layoutparser용 모델 로딩 (한 번만)
try:
    from layoutparser.models import Detectron2LayoutModel
    layout_model = Detectron2LayoutModel(
        config_path='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
        label_map={0: "Text"},
        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8]
    )
except Exception as e:
    layout_model = None
    print(f"❌ LayoutParser 모델 로딩 실패: {e}")

# 🔧 경로 설정
pdf_dir = Path("../data/no_upload")
pdf_files = list(pdf_dir.glob("*.pdf"))

# ✅ 순회 처리 함수
def run_test_on_all_pdfs():
    for file_path in pdf_files:
        print(f"\n📄 파일: {file_path.name}")

        try:
            plumber_result = extract_with_pdfplumber(str(file_path))
            print(f"✅ PDFPlumber 추출 성공 - 행 수: {len(plumber_result)}")
        except Exception as e:
            print(f"❌ PDFPlumber 실패: {e}")

        try:
            if layout_model is None:
                raise RuntimeError("layout_model 미로딩")
            layout_result = extract_with_layoutparser(str(file_path), layout_model)
            print(f"✅ LayoutParser 추출 성공 - 행 수: {len(layout_result)}")
        except Exception as e:
            print(f"❌ LayoutParser 실패: {e}")

# 🔁 테스트 실행
run_test_on_all_pdfs()


c:\Users\sokch\.conda\envs\ibk\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


❌ LayoutParser 모델 로딩 실패: 
Detectron2LayoutModel requires the detectron2 library but it was not found in your environment. Checkout the instructions on the
installation page: https://github.com/facebookresearch/detectron2/blob/master/INSTALL.md and follow the ones
that match your environment. Typically the following would work for MacOS or Linux CPU machines:
    pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2' 


📄 파일: 2205429_의사국 의안과_의안원문.pdf
✅ PDFPlumber 추출 성공 - 행 수: 0
❌ LayoutParser 실패: layout_model 미로딩

📄 파일: 2207157_의사국 의안과_의안원문.pdf
✅ PDFPlumber 추출 성공 - 행 수: 0
❌ LayoutParser 실패: layout_model 미로딩

📄 파일: 2208369_의사국 의안과_의안원문.pdf
✅ PDFPlumber 추출 성공 - 행 수: 0
❌ LayoutParser 실패: layout_model 미로딩

📄 파일: 2208659_의사국 의안과_의안원문.pdf
✅ PDFPlumber 추출 성공 - 행 수: 0
❌ LayoutParser 실패: layout_model 미로딩

📄 파일: 2208853_의사국 의안과_의안원문.pdf
✅ PDFPlumber 추출 성공 - 행 수: 0
❌ LayoutParser 실패: layout_model 미로딩

📄 파일: 2210255_의사국 의안과_의안원문.pdf
✅ PDFPlumber 추출 성공 - 행 수: 0
❌ La

In [5]:
from collections import defaultdict
import pdfplumber

def extract_with_pdfplumber(path, line_tol=1):
    result_rows = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            words = page.extract_words()
            if not words:
                continue

            midpoint = page.width / 2
            line_map = defaultdict(lambda: {'left': '', 'right': ''})

            for word in words:
                top_key = round(word['top'] / line_tol) * line_tol
                if word['x0'] < midpoint:
                    line_map[top_key]['left'] += ' ' + word['text']
                else:
                    line_map[top_key]['right'] += ' ' + word['text']

            for top in sorted(line_map.keys()):
                l = line_map[top]['left'].strip()
                r = line_map[top]['right'].strip()

                # ✅ 필터 조건 임시 제거
                # if not is_meaningless_dash_row(l, r): ← 제거
                result_rows.append([l, r])
    return result_rows


In [7]:
extract_with_pdfplumber(r"C:\Jimin\cg_DeltaLaw\data\no_upload\2210471_의사국 의안과_의안원문.pdf")

[['공휴일에 관한 법률', '일부개정법률안'],
 ['(김재섭의원', '대표발의)'],
 ['발의연월일', ': 2025. 5. 12.'],
 ['의 안', ''],
 ['10471', ''],
 ['번 호', ''],
 ['', '김재섭ㆍ백종헌ㆍ엄태영'],
 ['발', '의 자 :'],
 ['', '서천호ㆍ윤한홍ㆍ권영진'],
 ['', '강승규ㆍ최수진ㆍ신성범'],
 ['', '박덕흠 의원(10인)'],
 ['제안이유 및 주요내용', ''],
 ['저출산ㆍ고령사회', ''],
 ['「제4차 기본계획」은', '저출산의 원인과 관련하여'],
 ['일가정 양립을 저해하는 요인으로', '장시간 근로와 과소한 휴가일수를'],
 ['적시하고 있으나 유급 공휴일 도입에', '대하여는 관련 논의가 없음.'],
 ['5월 두 번째 금요일을 가족문화의', '날로 지정하여 가족친화적 휴일'],
 ['제도를 마련함으로써 일가정 양립을', '위한 휴가 필요성에 대한 인식을'],
 ['제고하려는 것임(안 제2조제6호 신설).', ''],
 ['- 1', '-'],
 ['법률 제 호', ''],
 ['공휴일에 관한 법률', '일부개정법률안'],
 ['공휴일에 관한 법률 일부를 다음과', '같이 개정한다.'],
 ['제2조제6호부터 제10호까지를 각각', '제7호부터 제11호까지로 하고, 같'],
 ['은 조에 제6호를 다음과 같이 신설한다.', ''],
 ['6. 가족문화의 날(5월 두 번째 금요일)', ''],
 ['부', '칙'],
 ['이 법은 공포한 날부터 시행한다.', ''],
 ['- 3', '-'],
 ['신ㆍ구조문대비표', ''],
 ['현 행', '개 정 안'],
 ['제2조(공휴일) 공휴일은 다음 각', '제2조(공휴일) --------------'],
 ['호와 같다.', '------------.'],
 ['1. ∼ 5. (생 략)', '1. ∼ 5. (현행과 같음)'],
 ['<신 설>', '6. 가족문화의 날(5월 두 번째'],
 ['', '금요일)'],
 [

In [8]:
import pdfplumber
from collections import defaultdict

def extract_with_pdfplumber_v2(path, line_tol=1):
    result_rows = []
    capturing = False

    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            if not capturing and ('신구조문대비표' in text or '현 행' in text):
                capturing = True

            if not capturing:
                continue

            words = page.extract_words()
            if not words:
                continue

            midpoint = page.width / 2
            line_map = defaultdict(lambda: {"left": "", "right": ""})

            for word in words:
                top_key = round(word['top'] / line_tol) * line_tol
                if word['x0'] < midpoint:
                    line_map[top_key]["left"] += " " + word['text']
                else:
                    line_map[top_key]["right"] += " " + word['text']

            for top in sorted(line_map.keys()):
                l = line_map[top]['left'].strip()
                r = line_map[top]['right'].strip()
                result_rows.append([l, r])

    # ✅ 후처리: 오른쪽 줄만 있는 경우 이전 행과 병합
    merged_rows = []
    for row in result_rows:
        if merged_rows and row[0] == "" and row[1]:
            merged_rows[-1][1] += " " + row[1]
        else:
            merged_rows.append(row)

    return merged_rows


In [9]:
extract_with_pdfplumber_v2(r"C:\Jimin\cg_DeltaLaw\data\no_upload\2210471_의사국 의안과_의안원문.pdf")

[['신ㆍ구조문대비표', ''],
 ['현 행', '개 정 안'],
 ['제2조(공휴일) 공휴일은 다음 각', '제2조(공휴일) --------------'],
 ['호와 같다.', '------------.'],
 ['1. ∼ 5. (생 략)', '1. ∼ 5. (현행과 같음)'],
 ['<신 설>', '6. 가족문화의 날(5월 두 번째 금요일)'],
 ['6. ∼ 10. (생 략)', '7. ∼ 11. (현행 제6호부터 제1 0호까지와 같음)'],
 ['- 5', '-']]

flowchart TD
    A[PDFPlumber로 줄 추출] --> B[좌우 병합 줄 리스트]
    B --> C[조문 시작 패턴으로 블록 분할]
    C --> D[좌우 블록 비교 → 변경유형 판정]
    D --> E[CSV + Excel로 저장]


### 전체 파일 순회 + 처리 + 저장

In [11]:
import pandas as pd
from pathlib import Path

def split_by_clause(rows):
    """조문 시작 기준으로 블록 단위 묶기"""
    def is_clause_start(text):
        return bool(re.match(r"^제?\d+조", text)) or bool(re.match(r"^[①-⑩\d]+\.", text))

    left_blocks, right_blocks = [], []
    cur_left, cur_right = [], []

    for l, r in rows:
        if is_clause_start(l):
            if cur_left or cur_right:
                left_blocks.append("\n".join(cur_left).strip())
                right_blocks.append("\n".join(cur_right).strip())
                cur_left, cur_right = [], []
            cur_left.append(l)
            cur_right.append(r)
        else:
            cur_left.append(l)
            cur_right.append(r)

    if cur_left or cur_right:
        left_blocks.append("\n".join(cur_left).strip())
        right_blocks.append("\n".join(cur_right).strip())

    return list(zip(left_blocks, right_blocks))

def classify_change_type(left, right):
    """변경유형 판정"""
    if left and not right:
        return "삭제"
    elif right and not left:
        return "신설"
    elif left == right:
        return "동일"
    else:
        import difflib
        sim = difflib.SequenceMatcher(None, left, right).ratio()
        return "동일" if sim >= 0.85 else "변경"

def process_pdf_file(file_path, out_dir):
    rows = extract_with_pdfplumber_v2(str(file_path))
    clause_pairs = split_by_clause(rows)
    data = []

    for left, right in clause_pairs:
        change_type = classify_change_type(left, right)
        data.append({
            "현행": left,
            "개정안": right,
            "변경유형": change_type
        })

    df = pd.DataFrame(data)
    law_name = file_path.stem
    df.to_csv(out_dir / f"{law_name}_조문비교결과.csv", index=False)
    df.to_excel(out_dir / f"{law_name}_조문비교결과.xlsx", index=False)

# 🔧 경로 설정
import re
pdf_dir = Path("../data/no_upload")
out_dir = Path("../data/processed")
out_dir.mkdir(parents=True, exist_ok=True)
pdf_files = list(pdf_dir.glob("*.pdf"))

# ✅ 전체 순회 실행
for file_path in pdf_files:
    print(f"📄 처리 중: {file_path.name}")
    process_pdf_file(file_path, out_dir)


📄 처리 중: 2205429_의사국 의안과_의안원문.pdf
📄 처리 중: 2207157_의사국 의안과_의안원문.pdf
📄 처리 중: 2208369_의사국 의안과_의안원문.pdf
📄 처리 중: 2208659_의사국 의안과_의안원문.pdf
📄 처리 중: 2208853_의사국 의안과_의안원문.pdf
📄 처리 중: 2210255_의사국 의안과_의안원문.pdf
📄 처리 중: 2210437_의사국 의안과_의안원문.pdf
📄 처리 중: 2210469_의사국 의안과_의안원문.pdf
📄 처리 중: 2210471_의사국 의안과_의안원문.pdf
📄 처리 중: 2210483_의사국 의안과_의안원문.pdf
📄 처리 중: 2210491_의사국 의안과_의안원문.pdf
📄 처리 중: 2210496_의사국 의안과_의안원문.pdf
📄 처리 중: 2210568_의사국 의안과_의안원문.pdf
📄 처리 중: 2210578_의사국 의안과_의안원문.pdf
📄 처리 중: 2210585_의사국 의안과_의안원문.pdf
📄 처리 중: 2210588_의사국 의안과_의안원문.pdf
📄 처리 중: file.pdf
📄 처리 중: file삭제있는것.pdf


In [12]:
import pandas as pd
import re
from pathlib import Path
from tqdm import tqdm

def split_by_clause(rows):
    def is_clause_start(text):
        return bool(re.match(r"^제?\d+조", text)) or bool(re.match(r"^[①-⑩\d]+\.", text))

    left_blocks, right_blocks = [], []
    cur_left, cur_right = [], []

    for l, r in rows:
        if is_clause_start(l):
            if cur_left or cur_right:
                left_blocks.append("\n".join(cur_left).strip())
                right_blocks.append("\n".join(cur_right).strip())
                cur_left, cur_right = [], []
            cur_left.append(l)
            cur_right.append(r)
        else:
            cur_left.append(l)
            cur_right.append(r)

    if cur_left or cur_right:
        left_blocks.append("\n".join(cur_left).strip())
        right_blocks.append("\n".join(cur_right).strip())

    return list(zip(left_blocks, right_blocks))

def classify_change_type(left, right):
    if left and not right:
        return "삭제"
    elif right and not left:
        return "신설"
    elif left == right:
        return "동일"
    else:
        import difflib
        sim = difflib.SequenceMatcher(None, left, right).ratio()
        return "동일" if sim >= 0.85 else "변경"

def process_pdf_and_append(file_path):
    rows = extract_with_pdfplumber_v2(str(file_path))
    clause_pairs = split_by_clause(rows)
    records = []

    for left, right in clause_pairs:
        change_type = classify_change_type(left, right)
        records.append({
            "파일명": file_path.name,
            "현행": left,
            "개정안": right,
            "변경유형": change_type
        })

    return records

# 🔧 경로
pdf_dir = Path("../data/no_upload")
out_dir = Path("../data/processed")
out_dir.mkdir(parents=True, exist_ok=True)
pdf_files = list(pdf_dir.glob("*.pdf"))

# ✅ 전체 순회 후 통합 저장
all_records = []
for file_path in tqdm(pdf_files, desc="PDF 처리 중"):
    all_records.extend(process_pdf_and_append(file_path))

df_all = pd.DataFrame(all_records)
df_all.to_csv(out_dir / "unit_test_조문_전체비교결과.csv", index=False)
df_all.to_excel(out_dir / "unit_test_조문_전체비교결과.xlsx", index=False)


PDF 처리 중: 100%|██████████| 18/18 [00:05<00:00,  3.30it/s]
